# DQN
## I. 用function approximation做TD learning of $V^{\pi}(S)$

### I.1 目标函数
$$J(w) = E\left [\left ( v^{\pi}(S)-\hat{v}(S,w)\right )^2 \right ]$$
$v^{\pi}(S)$和$\hat{v}(S,w)$分别是给定策略$\pi$时，所有state value的真实值和估计值。\
目标是找到参数w，满足：
$$
\begin{align}
w & = \underset{w}{argmin}J(w) \\
  & = \underset{w}{argmin} E\left [\left ( v^{\pi}(S)-\hat{v}(S,w)\right )^2 \right ]
\end{align}
$$
这里的期望是针对状态$s\in S$的分布而言。

### I.2 状态分布
#### I.2.1 状态转移概率矩阵
1. 定义状态转移概率矩阵: 假设每一步都有n种状态。  \
(1)单步状态转移概率矩阵：$P^{\pi}\in \mathbb{R}^{n\times n} $，其中单个元素为$[P^{\pi }]_{ij}$，表示当前状态是$s_i$时，下一步状态为$s_j$的概率。
$$[P^{\pi }]_{ij}=p^{\pi}(s_j|s_i)=\sum_{a\in \mathcal{A}}p(s_j|s_i, a)\pi(a|s_i)$$
(2)k步状态转移概率矩阵：$P^{\pi,(k)}\in \mathbb{R}^{n\times n}$，简记为$P^{(k)}$，其中单个元素$[P^{\pi,(k)}]_{ij}$（简记为$[P^{(k)}]_{ij}$）表示在策略$\pi$下，状态从$s_i$经过k步转移到$s_j$的概率。
$$
\begin{align}
[P^{\pi,(k)}]_{ij} & = [P^{(k)}]_{ij} = p^{\pi}(S_{t_k}=s_j|S_{t_0}=s_i) \\
[P^{\pi }]_{ij} & = [P^{(1)}]_{ij}
\end{align}
$$

2. **多步状态转移的概率可以表示为多个单步状态转移概率矩阵相乘。** \
根据经定义，两步时有：
$$
[(P^{\pi})^2]_{i, j}  = [P^{\pi}\times P^{\pi}]_{i, j}  = \sum_{h=1}^n [P^{\pi}]_{ih}\times [P^{\pi}]_{hj} = [P^{(2)}]_{i,j} 
$$
三步时有：
$$
\begin{align}
[(P^{\pi})^3]_{i, j} 
& = [(P^{\pi})^2\times P^{\pi}]_{i, j} \\
& = \sum_{h_2=1}^n \underset{p(s_i\rightarrow\forall s\rightarrow s_{h2})}{\underbrace{[(P^{\pi})^2]_{ih_2}}}\times [P^{\pi}]_{h_2j}  \\
& = \sum_{h_2=1}^n \underset{p(2步从s_i到s_{h2})}{\underbrace{[P^{(2)}]_{ih_2}}}\times  \underset{p(s_{h2}\rightarrow s_j)}{\underbrace{[P^{\pi}]_{h_2j}}}  \\
& =  \underset{p(3步从s_i\rightarrow s_j)}{\underbrace{[P^{(3)}]_{ij}}}\\
\end{align}
$$
扩展到多步：
$$
[(P^{\pi})^k]=[\underset{k个P^{\pi}相乘}{\underbrace{P^{\pi}\times P^{\pi}\times ... \times P^{\pi}}}  ]_{i, j}
= [P^{(k)}]_{i,j}
$$

#### I.2.2 定义状态的分布
1. $d_0\in \mathbb{R}^n$是n维向量，第i个元素$d_0(s_i)$表示起点时刻处于状态$s_i$的概率。
2. $d_k(s_i)$表示从起点开始，经过k步之后，agent处于状态$s_i$的概率。
$$
\begin{align}
d_k(s_i) & = \sum _{h = 1}^nd_0(s_h)[P^{(k)}]_{hi} \ ,\ i = 1, 2, ..., n \\
矩阵表达方式： \\
d_k^T & = d_0^TP^{(k)}
\end{align}$$
3. 状态分布的变化取决于状态转移概率
$$
d_k^T=d_{k-1}^TP^{(1)}=d_{k-1}^TP^{\pi}
$$

### I.3 状态分布的收敛
#### I.3.1 符号
$d^{\pi}(s)$表示agent用策略$\pi$经过很长一段时间后到达状态s的概率。
$$
d^{\pi}=\begin{pmatrix}
d^{\pi}(s_1) \\
d^{\pi}(s_2) \\
... \\
d^{\pi}(s_n)
\end{pmatrix}, 且\sum_{s\in S}d^{\pi}(s)=1
$$
#### I.3.2 两种收敛结果
1. **stationary distribution** \
stationary distribution刻画了马尔科夫过程的long-term behavior。在满足一定条件的情况下，当agent使用策略$\pi$与环境互动了sufficiently long period之后，state distribution $d_k$会收敛到constant value $d^{\pi}$。如果满足：
$$\therefore (d^{\pi})^T = (d^{\pi})^TP^{\pi }$$
就将$d^{\pi}$称为stationarity distribution。
2. **limiting distribution** \
如果满足：
$$\lim_{k \to \infty} d_k^T =(d^{\pi})^T$$
就将$d^{\pi}$称为limiting distribution。

3. <font color=red>两种收敛结果的区别：如果分布满足limiting distribution，则也就满足stationary distribution。但反之不然。</font> \
(1) 证明limiting distribution必然满足stationary distribution
<font color=grey>
$$
\begin{align}
&\because d_k^T  = d_{k-1}^TP^{\pi} \\
&\therefore \lim_{k \to \infty} d_k^T = \lim_{k \to \infty} d_{k-1}^TP^{\pi} \\
& 根据定义，如果是limiting\ distribution，则：\\
&\lim_{k \to \infty} d_k^T = (d^{\pi})^T,\lim_{k \to \infty} d_{k-1}^T= (d^{\pi})^T \\
&代入上式有： (d^{\pi})^T = (d^{\pi})^TP^{\pi } \\
\end{align}
$$
</font>
(2) stationary distribution也是limiting distribution的条件是：存在某个时刻k的状态分布能达到$d^{\pi}$，那么在k时刻之后的迭代必然一直停在$d^{\pi}$上。否则，虽然存在$d^{\pi}$满足$(d^{\pi})^T=(d^{\pi})^TP^{\pi}$，但不能保证agent一定能实现这样的状态分布。

#### I.3.3 状态分布收敛到limiting distribution
1. limiting distribution的<font color=blue>**充分条件**</font> \
如果在一定条件下，可以得到性质：
$$
\begin{align}
\lim_{k \to \infty} (P^{\pi })^k & = \mathbb{1}_n(d^{\pi})^T \\
&= \begin{pmatrix}
1 \\
1 \\
... \\
1
\end{pmatrix}(d^{\pi}(s_1), d^{\pi}(s_2),... ,d^{\pi}(s_n))\\
&=\begin{pmatrix}
d^{\pi}(s_1)  & d^{\pi}(s_2) & ... & d^{\pi}(s_n)\\
d^{\pi}(s_1)  & d^{\pi}(s_2) & ... & d^{\pi}(s_n)\\
...  &  &  & \\
d^{\pi}(s_1)  & d^{\pi}(s_2) & ... & d^{\pi}(s_n)
\end{pmatrix}
\end{align}
$$
证明：\
根据状态分布的定义，有：
$$
\lim_{k \to \infty} d_k^T =d_0^T\lim_{k \to \infty} (P^{\pi })^k= d_0^T\mathbb{1}_n(d^{\pi})^T = (d^{\pi})^T
$$

2. <font color=green>**能否收敛到limiting distribution，以及收敛后$d^{\pi}$的取值由状态转移概率矩阵$P^{\pi}$决定，与agent的初始状态无关。**</font> \
<font color=blue>注：状态转移概率矩阵$P^{\pi}$是由transition function和policy function决定的。</font> \
证明：
$$
\begin{align}
如果状态分布收敛： \\
&\because d_k^T  = d_{k-1}^TP^{\pi} \\
&\therefore \lim_{k \to \infty} d_k^T = \lim_{k \to \infty} d_{k-1}^TP^{\pi}= \left (\lim_{k \to \infty} d_{k-1}^T\right )P^{\pi} \\
&\therefore (d^{\pi})^T = (d^{\pi})^TP^{\pi } \\
&\therefore 0 = (d^{\pi})^T(P^{\pi }-I) \\
等价于求解： \\
&(P^{\pi }-I)^Td^{\pi} = 0 \\
& \left [(P^{\pi})^T-I \right]d^{\pi} = 0 \\
\end{align}
$$
所以$d^{\pi}$是矩阵$(P^{\pi})^T$的特征根，且对应的特征向量是1。

#### I.3.4 收敛条件：<font color=blue>irreducible or regular Markov process能够收敛到唯一的limiting distribution。</font>
1. 相关概念 \
(1) **accessible**: 如果$[P^{\pi}]^k_{ij}>0，且k是有限的，$则称从状态si到状态sj是accessible的。\
(2) **communicate**: 如果从状态si到状态sj是accessible的，且反过来从sj到si也是accessible的，那么就说si和sj是communicate的。\
(3) **irreducible**: 如果所有的状态pair都是communicate的，那么就称该马尔科夫过程是irreducible的。从数学定义上看，也就是对任意i，j，都存在$k\ge 1$使得$[P^k]_{ij}>0$ \
(4) **regular**: 如果存在$k\ge 1$使得对任意i，j都满足$[P^k]_{ij}>0$，则称该马尔科夫过程是regular的。可以简单表述为：存在k，使得$P^{\pi,(k)}>0$\
<font color=red>**注意irreducible和regular的区别**：irreducible只要求对于任意$s_i和s_j$，都能找到一个k，满足$[P^k]_{ij}>0$。因此每个状态pair找到的k值可以不同。regular要求存在同一个k，对任意$s_i和s_j$都满足$[P^k]_{ij}>0$。</font>

2. regular和irreducible相关的性质 \
(1) 如果一个马尔科夫过程是regular的，那么它必然是irreducible的。(显而易见) \
(2) 如果一个马尔科夫过程是irreducible的，且存在状态$s_i$满足$[p^{\pi}]_{ii}>0$，那么该马尔科夫过程就是regular的。\
（证略）\
(3) 如果$P^{\pi,(k)}>0$，那么对$\forall k^{'}> k$都满足$P^{\pi,(k^{'})}>0$。\
<font color=grey>简单证明：因为马尔科夫过程满足regular时，$P^{\pi}$不可能有全为0的列向量。</font> \
(4) 当马尔科夫过程是regular的，如果$\lim_{k \to \infty} (P^{\pi })^k = \mathbb{1}_n(d^{\pi})^T$，则对所有状态s都有$d^{\pi}(s)>0$。 \
<font color=grey>证明：
$$
\begin{align}
& \because \lim_{k \to \infty} (P^{\pi })^k = \mathbb{1}_n(d^{\pi})^T \\
& \therefore \lim_{k \to \infty} d_k^T = d_0^T\lim_{k \to \infty}(P^{\pi})^k = d_0^T\mathbb{1}_n(d^{\pi})^T= (d^{\pi})^T \\
& \because (P^{\pi})^k > 0 , 且d_0不会所有元素都为0\\
& \therefore (d^{\pi})^T > 0 
\end{align}
$$
</font>

#### I.3.5 什么样的策略能实现regular Markov process
<font color=blue>一般而言，exploratory policies比如$\epsilon-greedy$策略能够让马尔科夫过程具备regular特点。</font>\
说明：因为exploratory policies在任意时刻选择任意状态的概率都大于0。只要transition function允许，那么马尔科夫过程的所有状态pair都是communicate的。也因此，该马尔科夫过程是irreducible的。

### I.4 目标函数的优化
$$J(w) = E\left [\left ( v^{\pi}(S)-\hat{v}(S,w)\right )^2 \right ]$$


## II. 用function approximation做TD learning of $Q^{\pi}(S,A)$

## III. Deep Q-learning